In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
dflojas = pd.read_csv("lojas.csv")
dftreino = pd.read_csv("dataset_treino.csv")
dfteste = pd.read_csv("dataset_teste.csv")

In [3]:
#Searching for NAN
dftreino[dftreino.isna().values]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday


In [4]:
dfteste[dfteste.isna().values]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
479,480,622,4,2015-09-17,NaN,1,0,0
1335,1336,622,3,2015-09-16,NaN,1,0,0
2191,2192,622,2,2015-09-15,NaN,1,0,0
3047,3048,622,1,2015-09-14,NaN,1,0,0
4759,4760,622,6,2015-09-12,NaN,0,0,0
5615,5616,622,5,2015-09-11,NaN,0,0,0
6471,6472,622,4,2015-09-10,NaN,0,0,0
7327,7328,622,3,2015-09-09,NaN,0,0,0
8183,8184,622,2,2015-09-08,NaN,0,0,0
9039,9040,622,1,2015-09-07,NaN,0,0,0


In [5]:
dfteste.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [10]:
dfteste.groupby('Open').count()

,Id,Store,DayOfWeek,Date,Promo,StateHoliday,SchoolHoliday
Open,,,,,,,
0.0,5984,5984,5984,5984,5984,5984,5984
1.0,35093,35093,35093,35093,35093,35093,35093


In [11]:
dfteste.groupby('DayOfWeek').count()

,Id,Store,Date,Open,Promo,StateHoliday,SchoolHoliday
DayOfWeek,,,,,,,
1,5992,5992,5992,5990,5992,5992,5992
2,5992,5992,5992,5990,5992,5992,5992
3,5992,5992,5992,5990,5992,5992,5992
4,5992,5992,5992,5990,5992,5992,5992
5,5136,5136,5136,5135,5136,5136,5136
6,5992,5992,5992,5990,5992,5992,5992
7,5992,5992,5992,5992,5992,5992,5992


In [17]:
#Preencher Opens NAs com o valor de Promo (parece fazer sentido ter promoção com a loja aberta)
dfteste[dfteste.isna().values]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
479,480,622,4,2015-09-17,NaN,1,0,0
1335,1336,622,3,2015-09-16,NaN,1,0,0
2191,2192,622,2,2015-09-15,NaN,1,0,0
3047,3048,622,1,2015-09-14,NaN,1,0,0
4759,4760,622,6,2015-09-12,NaN,0,0,0
5615,5616,622,5,2015-09-11,NaN,0,0,0
6471,6472,622,4,2015-09-10,NaN,0,0,0
7327,7328,622,3,2015-09-09,NaN,0,0,0
8183,8184,622,2,2015-09-08,NaN,0,0,0
9039,9040,622,1,2015-09-07,NaN,0,0,0


In [34]:
for idx in dfteste[dfteste.isna().values]['Open'].index:
    dfteste['Open'][idx] = dfteste['Promo'][idx]

In [36]:
dfteste[dfteste.isna().values]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday


In [37]:
dfteste['Customers'] = dftreino['Customers'].mean()
dfteste.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Customers
0,1,1,4,2015-09-17,1.0,1,0,0,633.145946
1,2,3,4,2015-09-17,1.0,1,0,0,633.145946
2,3,7,4,2015-09-17,1.0,1,0,0,633.145946
3,4,8,4,2015-09-17,1.0,1,0,0,633.145946
4,5,9,4,2015-09-17,1.0,1,0,0,633.145946


In [38]:
dftreino.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday'],
      dtype='object')

In [45]:
dftreino['Year'] = pd.to_datetime(dftreino['Date']).apply(lambda data: data.year)
dftreino['Month'] = pd.to_datetime(dftreino['Date']).apply(lambda data: data.month)
dftreino['Day'] = pd.to_datetime(dftreino['Date']).apply(lambda data: data.day)

dfteste['Year'] = pd.to_datetime(dfteste['Date']).apply(lambda data: data.year)
dfteste['Month'] = pd.to_datetime(dfteste['Date']).apply(lambda data: data.month)
dfteste['Day'] = pd.to_datetime(dfteste['Date']).apply(lambda data: data.day)

In [47]:
def strToInt(x):
    if x == 0 or x == '0':
        return 0
    elif x == 'a':
        return 1
    elif x == 'b':
        return 2
    elif x == 'c':
        return 3
    else:
        return 4

dftreino['StateHoliday'] = dftreino['StateHoliday'].apply(lambda x: strToInt(x))
dfteste['StateHoliday'] = dfteste['StateHoliday'].apply(lambda x: strToInt(x))

In [46]:
dftreino.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,7,31
1,2,5,2015-07-31,6064,625,1,1,0,1,2015,7,31
2,3,5,2015-07-31,8314,821,1,1,0,1,2015,7,31
3,4,5,2015-07-31,13995,1498,1,1,0,1,2015,7,31
4,5,5,2015-07-31,4822,559,1,1,0,1,2015,7,31


In [39]:
def performance_metric(y_true, y_predict):
    """ Calcular e retornar a pontuação de desempenho entre 
        valores reais e estimados baseado na métrica escolhida. """
    
    # TODO: Calcular a pontuação de desempenho entre 'y_true' e 'y_predict'
    score = r2_score(y_true, y_predict)
    
    # Devolver a pontuação
    return score

In [41]:
def fit_model(X, y):
    """ Desempenhar busca em matriz sobre o parâmetro the 'max_depth' para uma 
        árvore de decisão de regressão treinada nos dados de entrada [X, y]. """
    
    # Gerar conjuntos de validação-cruzada para o treinamento de dados
    cv_sets = ShuffleSplit(n_splits=10, test_size = 0.20, random_state=0)

    # TODO: Gerar uma árvore de decisão de regressão de objeto
    regressor = DecisionTreeRegressor()

    # TODO: Gerar um dicionário para o parâmetro 'max_depth' com um alcance de 1 a 10
    params = {'max_depth': list(range(1, 11))}

    # TODO: Transformar 'performance_metric' em uma função de pontuação utilizando 'make_scorer' 
    scoring_fnc = make_scorer(performance_metric)

    # TODO: Gerar o objeto de busca em matriz
    grid = GridSearchCV(estimator=regressor, param_grid=params,scoring=scoring_fnc,cv=cv_sets)

    # Ajustar o objeto de busca em matriz com os dados para calcular o modelo ótimo
    grid = grid.fit(X, y)

    # Devolver o modelo ótimo depois de realizar o ajuste dos dados
    return grid.best_estimator_

In [49]:
X = dftreino[['Store', 'DayOfWeek', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Day']]
y = dftreino['Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [50]:
reg = fit_model(X_train, y_train)

In [51]:
print("O parâmetro 'max_depth' é {} para o modelo ótimo.".format(reg.get_params()['max_depth']))

O parâmetro 'max_depth' é 10 para o modelo ótimo.


In [52]:
predictions = reg.predict(X_test)

In [53]:
score = r2_score(y_test, predictions)
score

0.8903918931672892

In [58]:
dfteste.head(30)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Customers,Year,Month,Day
0,1,1,4,2015-09-17,1.0,1,0,0,633.145946,2015,9,17
1,2,3,4,2015-09-17,1.0,1,0,0,633.145946,2015,9,17
2,3,7,4,2015-09-17,1.0,1,0,0,633.145946,2015,9,17
3,4,8,4,2015-09-17,1.0,1,0,0,633.145946,2015,9,17
4,5,9,4,2015-09-17,1.0,1,0,0,633.145946,2015,9,17
5,6,10,4,2015-09-17,1.0,1,0,0,633.145946,2015,9,17
6,7,11,4,2015-09-17,1.0,1,0,0,633.145946,2015,9,17
7,8,12,4,2015-09-17,1.0,1,0,0,633.145946,2015,9,17
8,9,13,4,2015-09-17,1.0,1,0,0,633.145946,2015,9,17
9,10,14,4,2015-09-17,1.0,1,0,0,633.145946,2015,9,17


In [54]:
XComp = dfteste[['Store', 'DayOfWeek', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Day']]
predictions_comp = reg.predict(XComp)

In [55]:
predictions_comp

array([6485.61243221, 6485.61243221, 6485.61243221, ..., 6200.9992242 ,
       6200.9992242 , 6200.9992242 ])

In [56]:
dfPreditoTree = pd.DataFrame(predictions_comp)
dfPreditoTree.index = np.arange(1, len(dfPreditoTree)+1)
dfPreditoTree.to_csv('submissionTree0104.csv')